In [1]:
%run stdPackages.ipynb
d['data'] = os.path.join(d['curr'],'Misc','Data')
read = {'variable2D': ['data']}
db = dbFromWB(os.path.join(d['data'],'Correlations.xlsx'), read)

## Create representative samples and correlations

In [2]:
df = db['dataVariation'].unstack('data')
varnames = df.columns.copy()

## Create representative samples and correlations

Create indices corresponding to:
* Aggregate year to 24 hours.
* Aggregate year to 4 seasons, 24 hours.
* 6 hour intervals for each day:

*Hours:*

In [3]:
df['h_24hours'] = [i for i in range(1,25)]*365 

*Seasons (northern hemisphere)*

In [4]:
spring0 = (31+28)*24+1
springE = spring0+(31+30+31)*24
summer0 = springE+1
summerE = summer0+(30+31+31)*24
fall0   = summerE+1
fallE   = fall0+(30+31+30)*24
df['seasons'] = 'winter'
df['seasons'].iloc[spring0:springE] = 'spring'
df['seasons'].iloc[summer0:summerE] = 'summer'
df['seasons'].iloc[fall0:fallE] = 'fall'

C:\Users\sxj477\Anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


*6 and 3 hour intervals - 4 daily states and 6 daily states:*

In [5]:
df['h_4hours'] = np.hstack([[i]*6 for i in range(1,5)] * 365)
df['h_6hours'] = np.hstack([[i]*4 for i in range(1,7)] * 365)

#### Correlations:

In [6]:
corr = df[varnames].astype(float).corr()
db['correlations'] = corr.stack().rename_axis(['data','data_alias'])

#### Samples:

In [7]:
df = df.reset_index().set_index(['hdata','h_24hours','seasons','h_4hours','h_6hours'])

Create 24 hours representative states:

In [8]:
x = df[['LoadVariation_E','LoadVariation_H']].groupby(['h_24hours']).sum().rename_axis('h')
[db.__setitem__(k+'_24h', x[k]) for k in ['LoadVariation_E','LoadVariation_H']];
db['CapVariation_24h'] = df[['WS','PV','SH']].groupby('h_24hours').mean().stack().rename_axis(index={'h_24hours': 'h', 'data':'hvt'})

Create 4 representative states:

In [9]:
x = df[['LoadVariation_E','LoadVariation_H']].groupby(['h_4hours']).sum().rename_axis('h')
[db.__setitem__(k+'_4h', x[k]) for k in ['LoadVariation_E','LoadVariation_H']];
db['CapVariation_4h'] = df[['WS','PV','SH']].groupby('h_4hours').mean().stack().rename_axis(index={'h_4hours': 'h', 'data':'hvt'})

Create 6 representative states:

In [10]:
x = df[['LoadVariation_E','LoadVariation_H']].groupby(['h_6hours']).sum().rename_axis('h')
[db.__setitem__(k+'_6h', x[k]) for k in ['LoadVariation_E','LoadVariation_H']];
db['CapVariation_6h'] = df[['WS','PV','SH']].groupby('h_6hours').mean().stack().rename_axis(index={'h_6hours': 'h', 'data':'hvt'})

In [15]:
db['correlations'].unstack()

data_alias,LoadVariation_H,LoadVariation_E,WS,PV,SH
data,,,,,
LoadVariation_H,1.000000,0.322321,0.340809,-0.268178,-0.351990
LoadVariation_E,0.322321,1.000000,0.136334,0.215353,-0.085645
WS,0.340809,0.136334,1.000000,-0.211031,-0.180001
PV,-0.268178,0.215353,-0.211031,1.000000,0.493722
SH,-0.351990,-0.085645,-0.180001,0.493722,1.000000
